# Azure AI Foundry Agent Service - Python クイックスタート

このノートブックは、Azure AI Foundry Agent Service を使用してAIエージェントを作成・実行する方法を紹介します。最新の Azure AI Projects SDK（v1.0.0b12+）に対応し、APIの変更点やトラブルシューティング方法も含めて説明します。

## 📋 前提条件

- Azure サブスクリプション
- [Azure AI Foundry プロジェクト](https://ai.azure.com/?cid=learnDocs) (作成済み)
- プロジェクトスコープで **Azure AI User** RBAC ロールが割り当てられている
- Azure CLI がインストール済みで `az login` でサインイン済み

## 🛠️ 環境設定

プロジェクトルートの `.env` ファイルに以下を設定：

```bash
# Azure AI Foundry プロジェクトエンドポイント（必須）
PROJECT_ENDPOINT=https://YOUR-RESOURCE.services.ai.azure.com/api/projects/YOUR-PROJECT

# Azure AI Foundry プロジェクト名（必須）
AZURE_AI_PROJECT_NAME=YOUR-PROJECT-NAME

# Azure リソース情報（必須）
AZURE_SUBSCRIPTION_ID=YOUR-SUBSCRIPTION-ID
AZURE_OPENAI_RESOURCE_GROUP=YOUR-RESOURCE-GROUP-NAME

# モデルデプロイメント名（オプション - デフォルト: gpt-4o-mini）
MODEL_DEPLOYMENT_NAME=gpt-4o-mini
```

## 🔧 重要な注意点

⚠️ **Azure AI Projects SDK v1.0.0b12+ の重要な変更点**

### 📚 公式ドキュメント・リポジトリ
最新のSDK情報と変更点については、**必ず**以下の公式ソースを参照してください：

- **[MS Learn: Azure AI Projects SDK Python リファレンス](https://learn.microsoft.com/ja-jp/python/api/overview/azure/ai-projects-readme)**
- **[MS Learn: Azure AI Foundry クイックスタート](https://learn.microsoft.com/azure/ai-foundry/quickstarts/get-started-code)**
- **[GitHub: Azure SDK for Python - AI Projects](https://github.com/Azure/azure-sdk-for-python/tree/main/sdk/ai/azure-ai-projects)**
- **[GitHub: サンプルコード](https://github.com/Azure/azure-sdk-for-python/tree/main/sdk/ai/azure-ai-projects/samples)**

### 1. シンプルな初期化
- **v1.0.0b12+** では、`endpoint` と `credential` のみで初期化可能
- 以前のバージョンで必須だった `subscription_id`, `resource_group_name`, `project_name` は不要

### 2. API構造の変更
- ✅ **新API**: `project_client.agents.threads.create()` 
- ❌ **旧API**: `project_client.agents.create_thread()` (廃止)
- ✅ **新API**: `project_client.agents.messages.create()`
- ❌ **旧API**: `project_client.agents.create_message()` (廃止)
- ✅ **新API**: `project_client.agents.runs.create_and_process()`

### 3. ItemPaged オブジェクトの変更
- ✅ **推奨**: 直接イテレーション `for msg in messages:`
- ❌ **廃止**: `.data` 属性 (`messages.data` は使用不可)
- ✅ **代替**: `list(messages)` で変換

### 4. 環境変数の統一
このノートブックは、リポジトリ全体で使用されている標準的な環境変数名を採用しています：

- `AZURE_AI_PROJECT_NAME` (プロジェクト名)
- `AZURE_OPENAI_RESOURCE_GROUP` (リソースグループ名)  
- `MODEL_DEPLOYMENT_NAME` (モデルデプロイメント名)
- `PROJECT_ENDPOINT` (プロジェクトエンドポイント)
- `AZURE_SUBSCRIPTION_ID` (サブスクリプション ID)

これにより、他の章のサンプルコードとの一貫性を保ちます。

> ⚠️ **重要**: SDKのバージョンアップに伴う変更は頻繁に発生します。エラーが発生した場合は、上記の公式ドキュメントで最新のAPIリファレンスと使用方法を必ず確認してください。

In [ ]:
# 1. セットアップと認証
import os
import sys
import subprocess

# 必要なパッケージをインストール/アップデート
print("📦 必要なパッケージを確認・アップデート中...")
try:
    # 最新バージョンのazure-ai-projectsをインストール
    subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", 
                          "azure-ai-projects", "azure-identity", "python-dotenv", "openai", "--quiet"])
    print("✅ パッケージのアップデートが完了しました")
except subprocess.CalledProcessError as e:
    print(f"⚠️ パッケージのインストール/アップデートに失敗: {e}")
    raise

# パッケージをインポート
try:
    import azure.ai.projects
    import azure.identity
    import dotenv
    import openai
    print("✅ 必要なパッケージのインポートが完了しました")
except ImportError as e:
    print(f"❌ パッケージのインポートに失敗: {e}")
    print("Jupyter カーネルを再起動してください")
    raise

# バージョン情報を表示
print(f"📋 パッケージバージョン:")
print(f"• azure-ai-projects: {azure.ai.projects.__version__}")
print(f"• azure-identity: {azure.identity.__version__}")
print(f"• openai: {openai.__version__}")

# 環境変数を読み込み
from dotenv import load_dotenv
load_dotenv()

# .env ファイルで定義された環境変数を取得（リポジトリ標準の変数名）
PROJECT_ENDPOINT = os.getenv("PROJECT_ENDPOINT")
AZURE_AI_PROJECT_NAME = os.getenv("AZURE_AI_PROJECT_NAME")
# 新しい標準的な変数名を優先し、古い名前での後方互換性も提供
MODEL_DEPLOYMENT_NAME = os.getenv("MODEL_DEPLOYMENT_NAME") or os.getenv("AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME", "gpt-4o-mini")
AZURE_SUBSCRIPTION_ID = os.getenv("AZURE_SUBSCRIPTION_ID")
AZURE_OPENAI_RESOURCE_GROUP = os.getenv("AZURE_OPENAI_RESOURCE_GROUP")

print("\n🔍 環境変数の確認:")
print(f"PROJECT_ENDPOINT: {'✅ 設定済み' if PROJECT_ENDPOINT else '❌ 未設定'}")
print(f"AZURE_AI_PROJECT_NAME: {'✅ 設定済み' if AZURE_AI_PROJECT_NAME else '❌ 未設定'}")
print(f"MODEL_DEPLOYMENT_NAME: {MODEL_DEPLOYMENT_NAME}")
print(f"AZURE_SUBSCRIPTION_ID: {'✅ 設定済み' if AZURE_SUBSCRIPTION_ID else '❌ 未設定'}")
print(f"AZURE_OPENAI_RESOURCE_GROUP: {'✅ 設定済み' if AZURE_OPENAI_RESOURCE_GROUP else '❌ 未設定'}")

# 必須環境変数チェック
missing_vars = []
if not PROJECT_ENDPOINT:
    missing_vars.append("PROJECT_ENDPOINT")
if not AZURE_AI_PROJECT_NAME:
    missing_vars.append("AZURE_AI_PROJECT_NAME")
if not AZURE_SUBSCRIPTION_ID:
    missing_vars.append("AZURE_SUBSCRIPTION_ID")
if not AZURE_OPENAI_RESOURCE_GROUP:
    missing_vars.append("AZURE_OPENAI_RESOURCE_GROUP")

if missing_vars:
    print(f"\n❌ エラー: 以下の環境変数が設定されていません: {', '.join(missing_vars)}")
    print("\n📝 .env ファイルに以下を追加してください:")
    for var in missing_vars:
        if var == "PROJECT_ENDPOINT":
            print("PROJECT_ENDPOINT=https://YOUR-RESOURCE.services.ai.azure.com/api/projects/YOUR-PROJECT")
        elif var == "AZURE_AI_PROJECT_NAME":
            print("AZURE_AI_PROJECT_NAME=YOUR-PROJECT-NAME")
        elif var == "AZURE_SUBSCRIPTION_ID":
            print("AZURE_SUBSCRIPTION_ID=YOUR-SUBSCRIPTION-ID")
        elif var == "AZURE_OPENAI_RESOURCE_GROUP":
            print("AZURE_OPENAI_RESOURCE_GROUP=YOUR-RESOURCE-GROUP-NAME")
    print("\n📖 詳細: https://learn.microsoft.com/azure/ai-foundry/quickstarts/get-started-code")
    raise ValueError("必要な環境変数が設定されていません")

print("\n✅ 環境変数の設定確認が完了しました")
print("💡 リポジトリ標準の環境変数名を使用：統一性とメンテナンス性を向上")

In [ ]:
# Azure AI Foundry プロジェクトクライアント設定
import os
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient

print("🔐 Azure AI Foundry プロジェクトに接続中...")

# 認証設定（Azure CLI認証）
credential = DefaultAzureCredential()

print("\n🔑 認証設定:")
print("• 認証方法: DefaultAzureCredential (Azure CLI認証)")

# 環境変数をリポジトリ標準の名前で取得
required_vars = {
    'PROJECT_ENDPOINT': PROJECT_ENDPOINT,
    'MODEL_DEPLOYMENT_NAME': MODEL_DEPLOYMENT_NAME,
    'AZURE_SUBSCRIPTION_ID': AZURE_SUBSCRIPTION_ID,
    'AZURE_OPENAI_RESOURCE_GROUP': AZURE_OPENAI_RESOURCE_GROUP,
    'AZURE_AI_PROJECT_NAME': AZURE_AI_PROJECT_NAME
}

print("\n🔍 環境変数の確認:")
missing_vars = []
for var_name, var_value in required_vars.items():
    if var_value:
        display_value = var_value[:20] + "..." if len(var_value) > 20 else var_value
        print(f"✅ {var_name}: {display_value}")
    else:
        print(f"❌ {var_name}: 未設定")
        missing_vars.append(var_name)

if missing_vars:
    print(f"\n❌ エラー: 以下の環境変数が設定されていません: {', '.join(missing_vars)}")
    print("\n📝 解決方法:")
    print("1. プロジェクトルートの .env ファイルを開く")
    print("2. 以下の環境変数を追加・設定:")
    for var in missing_vars:
        if var == 'AZURE_OPENAI_RESOURCE_GROUP':
            print(f"   AZURE_OPENAI_RESOURCE_GROUP=your-resource-group-name")
        elif var == 'AZURE_AI_PROJECT_NAME':
            print(f"   AZURE_AI_PROJECT_NAME=your-project-name")
        elif var == 'PROJECT_ENDPOINT':
            print(f"   PROJECT_ENDPOINT=https://your-resource.services.ai.azure.com/api/projects/your-project")
        elif var == 'AZURE_SUBSCRIPTION_ID':
            print(f"   AZURE_SUBSCRIPTION_ID=your-subscription-id")
        elif var == 'MODEL_DEPLOYMENT_NAME':
            print(f"   MODEL_DEPLOYMENT_NAME=gpt-4o-mini")
        else:
            print(f"   {var}=your-{var.lower().replace('_', '-')}")
    print("3. VS Code/Jupyter カーネルを再起動")
    print("4. セルを再実行")
    print("\n🔗 詳細なトラブルシューティング: azureaiagent_troubleshooting.md ファイルを参照")
    raise ValueError("必要な環境変数が設定されていません")

# プロジェクトクライアントを作成
print("\n🚀 AIProjectClient を作成中...")

try:
    # v1.0.0b12+ 推奨方法: endpoint + credential のみ
    print("• 最新SDK方式: endpoint + credential のみ")
    project_client = AIProjectClient(
        endpoint=required_vars['PROJECT_ENDPOINT'],
        credential=credential
    )
    print("✅ プロジェクトクライアント作成が成功しました")
    
except Exception as e1:
    print(f"• 最新SDK方式失敗: {e1}")
    
    # TypeErrorの場合は、SDKバージョンが古い可能性を示唆
    if "missing" in str(e1) and "required positional arguments" in str(e1):
        print("\n💡 このエラーは古いSDKバージョンを示している可能性があります")
        print("   最新のAzure AI Projects SDK (v1.0.0b12+) では、endpointとcredentialのみで動作します")
        print("   詳細は以下を参照してください:")
        print("   https://learn.microsoft.com/en-us/python/api/overview/azure/ai-projects-readme")
        
    print(f"\n❌ プロジェクトクライアントの作成に失敗しました")
    print(f"\n🔍 エラータイプ別の解決方法:")
    print(f"1. 'missing required positional arguments' エラー:")
    print(f"   → SDKバージョンを確認・アップグレード: pip install --upgrade azure-ai-projects")
    print(f"2. 'Authentication failed' エラー:")
    print(f"   → Azure CLI で 'az login' を実行してサインイン")
    print(f"3. 'Resource not found' エラー:")
    print(f"   → プロジェクト名・リソースグループ名・サブスクリプション ID・エンドポイントを確認")
    print(f"4. 'Permission denied' エラー:")
    print(f"   → プロジェクトで 'Azure AI User' RBAC ロールが割り当てられているか確認")
    print(f"5. 'Invalid endpoint' エラー:")
    print(f"   → PROJECT_ENDPOINT の形式を確認: https://your-resource.services.ai.azure.com/api/projects/your-project")
    print(f"\n🔗 詳細なトラブルシューティング: azureaiagent_troubleshooting.md ファイルを参照")
    raise e1

print("\n✅ Azure AI Foundry プロジェクトクライアントの作成完了")
print(f"• プロジェクトエンドポイント: {required_vars['PROJECT_ENDPOINT'][:50]}...")
print(f"• モデルデプロイメント: {required_vars['MODEL_DEPLOYMENT_NAME']}")
print(f"• SDK バージョン: {azure.ai.projects.__version__}")

In [ ]:
# 2. SDK調査・テスト（オプション）
# 注意: このセルはSDKの動作確認用です。実際のエージェント実行はセル5で行います。
# project_clientオブジェクトの属性とメソッドを確認
print("🔍 project_client のメソッド:")
print([attr for attr in dir(project_client) if not attr.startswith('_')])

print("\n🔍 project_client.agents のメソッド:")
print([attr for attr in dir(project_client.agents) if not attr.startswith('_')])

# help()でドキュメントを確認
help(project_client.agents)

# SDKの詳細調査
print("\n🔍 project_client.agents.threads のメソッド:")
print([attr for attr in dir(project_client.agents.threads) if not attr.startswith('_')])

print("\n🔍 project_client.agents.messages のメソッド:")
print([attr for attr in dir(project_client.agents.messages) if not attr.startswith('_')])

print("\n🔍 project_client.agents.runs のメソッド:")
print([attr for attr in dir(project_client.agents.runs) if not attr.startswith('_')])

# 新しいAPIでのスレッド作成をテスト
print("\n🧪 新しいAPIでスレッド作成をテスト:")
try:
    test_thread = project_client.agents.threads.create()
    print(f"✅ スレッド作成成功: {test_thread.id}")
    print(f"📋 スレッドオブジェクトの属性: {[attr for attr in dir(test_thread) if not attr.startswith('_')]}")
    
    # テストメッセージを作成
    print("\n🧪 テストメッセージ作成:")
    test_message = project_client.agents.messages.create(
        thread_id=test_thread.id,
        role="user",
        content="テストメッセージ"
    )
    print(f"✅ メッセージ作成成功: {test_message.id}")
    
    # messages.list()の戻り値の型を確認
    print("\n🧪 messages.list()の戻り値を調査:")
    messages_result = project_client.agents.messages.list(thread_id=test_thread.id)
    print(f"📋 messages.list()の戻り値の型: {type(messages_result)}")
    print(f"📋 messages.list()の戻り値の属性: {[attr for attr in dir(messages_result) if not attr.startswith('_')]}")
    
    # ItemPagedオブジェクトの使用方法を確認
    print("\n🔍 ItemPagedオブジェクトの使用方法:")
    print("方法1: リスト変換")
    try:
        messages_list = list(messages_result)
        print(f"✅ list()変換成功: {len(messages_list)} メッセージ")
        if messages_list:
            print(f"📋 最初のメッセージ: {messages_list[0].id}")
    except Exception as e:
        print(f"❌ list()変換失敗: {e}")
    
    print("\n方法2: 直接イテレーション")
    try:
        for i, msg in enumerate(messages_result):
            print(f"✅ メッセージ {i+1}: {msg.id} ({msg.role})")
            if i >= 2:  # 最初の3つだけ表示
                break
    except Exception as e:
        print(f"❌ 直接イテレーション失敗: {e}")
    
    print("\n方法3: .data属性の確認")
    if hasattr(messages_result, 'data'):
        print(f"✅ .data属性存在: {type(messages_result.data)}")
    else:
        print(f"❌ .data属性存在しない")
        
    print("\n方法4: by_page()メソッドの確認")
    if hasattr(messages_result, 'by_page'):
        print(f"✅ .by_page()メソッド存在")
        try:
            pages = messages_result.by_page()
            for page in pages:
                print(f"📋 ページ内容: {len(list(page))} アイテム")
                break
        except Exception as e:
            print(f"❌ .by_page()使用失敗: {e}")
    else:
        print(f"❌ .by_page()メソッド存在しない")
        
    # クリーンアップ
    print("\n🧹 テストリソースをクリーンアップ:")
    project_client.agents.threads.delete(test_thread.id)
    print(f"✅ テストスレッド削除完了: {test_thread.id}")
    
except Exception as e:
    print(f"❌ テスト失敗: {e}")
    print(f"エラータイプ: {type(e).__name__}")

# 詳細なドキュメントを確認
print("\n📖 messages.listメソッドのヘルプ:")
help(project_client.agents.messages.list)

In [ ]:
# 3. エージェントの作成・実行・クリーンアップ（最新SDK対応・ItemPaged対応）
# 注意: このセルは実際のエージェント実行を行います（セル4は調査用のテストセル）

# 出力をクリア（重複表示を防ぐ）
import IPython
if hasattr(IPython.get_ipython(), 'kernel'):
    try:
        from IPython.display import clear_output
        clear_output(wait=True)
    except:
        pass

print("🤖 Azure AI エージェントのワークフローを開始します...")

try:
    # ステップ1: エージェントを作成
    print("\n📋 エージェントを作成中...")
    agent = project_client.agents.create_agent(
        model=MODEL_DEPLOYMENT_NAME,
        name="Python Demo Agent",
        instructions="あなたは親切で知識豊富なアシスタントです。日本語で簡潔かつ分かりやすく回答してください。"
    )
    print(f"✅ エージェント作成完了: {agent.id}")

    # ステップ2: スレッド（会話セッション）を作成（新しいAPI）
    print("\n💬 会話スレッドを作成中...")
    thread = project_client.agents.threads.create()
    print(f"✅ スレッド作成完了: {thread.id}")

    # ステップ3: ユーザーメッセージを作成（新しいAPI）
    user_message = "Azure AI Foundry Agent Service について簡潔に教えてください。"
    print(f"\n📝 ユーザーメッセージ: {user_message}")
    
    message = project_client.agents.messages.create(
        thread_id=thread.id,
        role="user",
        content=user_message
    )
    print(f"✅ メッセージ作成完了: {message.id}")

    # ステップ4: エージェントを実行（新しいAPI）
    print(f"\n🏃 エージェントを実行中...")
    run = project_client.agents.runs.create_and_process(
        thread_id=thread.id,
        agent_id=agent.id
    )
    print(f"✅ エージェント実行完了: {run.id} (ステータス: {run.status})")

    # ステップ5: 会話履歴を取得・表示（ItemPaged対応）
    print(f"\n📋 会話履歴を取得中...")
    messages_paged = project_client.agents.messages.list(thread_id=thread.id)
    
    # ItemPagedオブジェクトを安全にリストに変換（複数の方法を試行）
    messages_list = []
    try:
        # 方法1: 直接イテレーション（推奨・最新SDK対応）
        for msg in messages_paged:
            messages_list.append(msg)
        print(f"✅ 直接イテレーションでメッセージ取得成功")
    except Exception as e:
        print(f"⚠️ 直接イテレーション失敗: {e}")
        try:
            # 方法2: list()変換（代替案）
            messages_list = list(messages_paged)
            print(f"✅ list()変換でメッセージ取得成功")
        except Exception as e2:
            print(f"⚠️ list()変換も失敗: {e2}")
            # 方法3: .data属性（古いSDKとの互換性）
            if hasattr(messages_paged, 'data'):
                messages_list = messages_paged.data
                print(f"✅ .data属性でメッセージ取得成功（旧SDK互換）")
            else:
                print(f"❌ メッセージ取得に失敗しました")
                raise e
    
    print(f"✅ メッセージ取得完了: {len(messages_list)} 件")
    
    print("\n" + "="*60)
    print("💬 エージェントとの会話")
    print("="*60)
    
    # メッセージを時系列順に表示（最新が最初に来るので逆順）
    for msg in reversed(messages_list):
        role_emoji = "🧑 ユーザー" if msg.role == "user" else "🤖 エージェント"
        
        # メッセージ内容を安全に取得
        try:
            if msg.content and len(msg.content) > 0:
                # 最新SDKでのメッセージ内容の取得
                content_item = msg.content[0]
                if hasattr(content_item, 'text'):
                    content = content_item.text.value
                elif hasattr(content_item, 'value'):
                    content = content_item.value
                else:
                    content = str(content_item)
            else:
                content = "(内容なし)"
        except Exception as e:
            content = f"(メッセージ内容取得エラー: {e})"
        
        print(f"\n{role_emoji}:")
        print(f"{content}")
    
    print("\n" + "="*60)

    # ステップ6: クリーンアップ（エージェントを削除）
    print(f"\n🧹 リソースをクリーンアップ中...")
    project_client.agents.delete_agent(agent.id)
    print(f"✅ エージェント削除完了: {agent.id}")
    
    print(f"\n🎉 Azure AI エージェントのデモンストレーションが正常に完了しました！")
    print(f"✨ SDK v{azure.ai.projects.__version__} での新APIを正常に使用")

except Exception as e:
    print(f"\n❌ エラーが発生しました: {str(e)}")
    print(f"\n🔍 エラーの詳細:")
    print(f"エラータイプ: {type(e).__name__}")
    
    # エラー時のみSDK調査を実行（メッセージ取得による重複出力を防ぐ）
    print(f"\n📋 SDK情報の調査:")
    try:
        # メッセージ取得せずに、オブジェクトの型情報のみ確認
        if 'messages_paged' in locals():
            print(f"戻り値の型: {type(messages_paged)}")
            print(f"利用可能な属性: {[attr for attr in dir(messages_paged) if not attr.startswith('_')]}")
            
            # 使用可能なメソッドを確認
            if hasattr(messages_paged, '__iter__'):
                print("✅ イテレーション可能")
            if hasattr(messages_paged, 'data'):
                print("✅ .data属性存在")
            if hasattr(messages_paged, 'by_page'):
                print("✅ .by_page()メソッド存在")
        else:
            print("messages_paged オブジェクトが利用できません")
            
    except Exception as e2:
        print(f"SDK情報調査失敗: {e2}")
    
    # 新しいSDKのメソッドを動的に確認
    print(f"\n📋 利用可能なメソッドを確認:")
    if hasattr(project_client.agents, 'threads'):
        print(f"✅ threads サブオブジェクト利用可能")
        print(f"   threads のメソッド: {[attr for attr in dir(project_client.agents.threads) if not attr.startswith('_')]}")
    
    if hasattr(project_client.agents, 'messages'):
        print(f"✅ messages サブオブジェクト利用可能")
        print(f"   messages のメソッド: {[attr for attr in dir(project_client.agents.messages) if not attr.startswith('_')]}")
    
    if hasattr(project_client.agents, 'runs'):
        print(f"✅ runs サブオブジェクト利用可能")
        print(f"   runs のメソッド: {[attr for attr in dir(project_client.agents.runs) if not attr.startswith('_')]}")
    
    print(f"\n🔍 トラブルシューティング:")
    print(f"1. Azure CLI で 'az login' を実行してサインインしているか確認")
    print(f"2. モデル '{MODEL_DEPLOYMENT_NAME}' がプロジェクトにデプロイされているか確認")
    print(f"3. プロジェクトで 'Azure AI User' RBAC ロールが割り当てられているか確認")
    print(f"4. PROJECT_ENDPOINT が正しく設定されているか確認")
    print(f"5. SDKの最新版を使用しているか確認（現在: {azure.ai.projects.__version__}）")
    print(f"6. ItemPagedオブジェクトの使用方法を確認（.data属性は廃止）")
    
    # エラー時のクリーンアップを試行
    try:
        if 'agent' in locals():
            project_client.agents.delete_agent(agent.id)
            print(f"⚠️ エラー後クリーンアップ: エージェント {agent.id} を削除しました")
    except:
        pass
    
    # 代替案を提案
    print(f"\n💡 代替案:")
    print(f"1. create_thread_and_run() メソッドを使用（スレッド作成と実行を一度に）")
    print(f"2. create_thread_and_process_run() メソッドを使用（スレッド作成と処理実行を一度に）")
    print(f"3. get_last_message_text_by_role() メソッドを使用（最新メッセージのテキスト取得）")
    
    raise

## 🎯 次のステップ

おめでとうございます！Azure AI Foundry Agent Service の基本的なワークフローを体験しました。

### 🎓 学習したこと

- ✅ Azure AI Projects SDK v1.0.0b12+ を使用したプロジェクト接続
- ✅ DefaultAzureCredential による Azure CLI 認証
- ✅ `.env` ファイルからの統一された環境変数読み込み
- ✅ エージェントの作成・設定・削除
- ✅ 新しいAPI構造でのスレッドとメッセージの管理
- ✅ エージェントの実行と応答の処理
- ✅ ItemPaged オブジェクトの正しい使用方法

### 🔧 重要な技術的ポイント

1. **最新SDKの要件**: `AIProjectClient` には全ての必須パラメータを明示的に指定
2. **堅牢な環境変数管理**: Azure CLI コマンドに依存しない設定
3. **新しいAPI構造**: `threads.create()`, `messages.create()`, `runs.create_and_process()`
4. **ItemPaged 対応**: `.data` 属性の廃止に対応した直接イテレーション
5. **エラーハンドリング**: 包括的なトラブルシューティングと代替案

### 🚀 応用例

1. **ツールの追加**: コードインタープリターやファイル検索ツールの統合
2. **継続的な会話**: 同じスレッドで複数メッセージのやり取り
3. **カスタム関数**: Function Calling でエージェントに独自機能を追加
4. **ファイル処理**: ドキュメントやデータファイルをエージェントに提供
5. **マルチエージェント**: 複数のエージェントによる協調作業

### 📚 リソース

- **[MS Learn: Azure AI Foundry Agent Service](https://learn.microsoft.com/ja-jp/azure/ai-foundry/agents/)**
- **[MS Learn: クイックスタート](https://learn.microsoft.com/azure/ai-foundry/quickstarts/get-started-code)**
- **[MS Learn: Entra ID認証](https://learn.microsoft.com/azure/ai-foundry/foundry-models/how-to/configure-entra-id)**
- **[Python SDK リファレンス](https://learn.microsoft.com/ja-jp/python/api/overview/azure/ai-projects-readme)**
- **[GitHub サンプルコード](https://github.com/Azure/azure-sdk-for-python/tree/main/sdk/ai/azure-ai-projects/samples)**
- **[エージェントプレイグラウンド](https://ai.azure.com/?cid=learnDocs)**

### 🔧 トラブルシューティング

よくある問題と解決方法：

1. **`missing required positional arguments` エラー**
   - `.env` ファイルの `AZURE_SUBSCRIPTION_ID`, `AZURE_OPENAI_RESOURCE_GROUP`, `AZURE_AI_PROJECT_NAME` を確認

2. **`Authentication failed` エラー**
   - Azure CLI で `az login` を実行してサインイン

3. **`Resource not found` エラー**
   - プロジェクト名・リソースグループ名・サブスクリプション ID を確認

4. **`Permission denied` エラー**
   - プロジェクトで 'Azure AI User' RBAC ロールが割り当てられているか確認

5. **`'AgentsClient' object has no attribute 'create_thread'` エラー**
   - 新しいAPIを使用: `project_client.agents.threads.create()` 

6. **`'ItemPaged' object has no attribute 'data'` エラー**
   - 直接イテレーション使用: `for msg in messages_paged:`

### 💡 SDKの重要な変更点（v1.0.0b12+）

⚠️ **破壊的変更に注意**:

| 旧API（廃止） | 新API（推奨） |
|---|---|
| `agents.create_thread()` | `agents.threads.create()` |
| `agents.create_message()` | `agents.messages.create()` |
| `agents.create_run()` | `agents.runs.create()` |
| `messages.data` | `for msg in messages:` |
| `endpoint` のみ | 全必須パラメータ指定 |

### 🌟 成功のポイント

1. **環境変数の統一**: リポジトリ全体で一貫した名前を使用
2. **SDKバージョン管理**: 最新バージョンへの適応
3. **エラー処理**: 包括的なトラブルシューティング
4. **後方互換性**: 複数の方法でのフォールバック実装

### 📖 参考資料

**最新の情報は必ず公式ドキュメントを参照してください：**
- [MS Learn: Azure AI Projects Python SDK](https://learn.microsoft.com/python/api/overview/azure/ai-projects-readme)
- [MS Learn: Azure AI Foundry 開発者ガイド](https://learn.microsoft.com/azure/ai-foundry/how-to/)
- [GitHub: Azure SDK for Python - AI Projects](https://github.com/Azure/azure-sdk-for-python/tree/main/sdk/ai/azure-ai-projects)
- [GitHub: サンプルコードとチュートリアル](https://github.com/Azure/azure-sdk-for-python/tree/main/sdk/ai/azure-ai-projects/samples)

詳細なトラブルシューティング情報は `azureaiagent_troubleshooting.md` を参照してください。